In [1]:
import asyncio
import numpy as np
import quaternion

from intrepid_environment.simulator import Simulator

In [2]:
import matplotlib.pyplot as plt

In [3]:
import nest_asyncio
nest_asyncio.apply()

In [4]:
sim = Simulator()
await sim.connect()

await sim.rpc("session.restart")
entity = await sim.rpc("map.spawn_uav", {
    "robot_id": 0,
    "position": {
        "x": 0,
        "y": 0,
        "z": 0,
    },
})
await sim.rpc("session.run")

Setup trajectory for drone to follow.

In [5]:
from quadcopter_control.trajectory import Trajectory, TrajectoryPoint

In [6]:
async def sim_step(motors):

    (sim_time, _, drone_state) = await asyncio.gather(
            sim.step(),
            sim.rpc(f"object_{entity}.actuator_control", motors),
            sim.rpc("script.eval", {
                "code": """{
                    position = sim.object.position(ARGS),
                    attitude = sim.object.rotation_quat(ARGS),
                    velocity = sim.object.linear_velocity(ARGS),
                    omega = sim.object.angular_velocity(ARGS),
                    acceleration = sim.object.acceleration(ARGS),
                }""",
                "args": entity,
            }),
        )

    acceleration = np.array([drone_state['acceleration']['y'],
                             drone_state['acceleration']['x'],
                             drone_state['acceleration']['z']], dtype=np.float64)

    position = np.array([drone_state['position']['y'],
                        drone_state['position']['x'],
                        drone_state['position']['z']], dtype=np.float64)

    velocity = np.array([drone_state['velocity']['y'],
                        drone_state['velocity']['x'],
                        drone_state['velocity']['z']], dtype=np.float64)

    attitude = np.quaternion(drone_state['attitude'][3],
                             drone_state['attitude'][0],
                             drone_state['attitude'][1],
                             drone_state['attitude'][2])

    omega = np.array([drone_state['omega']['xy'],
                     drone_state['omega']['zx'],
                     drone_state['omega']['yz']], dtype=np.float64)

    return {
        'time': sim_time,
        'acceleration': acceleration,
        'position': position,
        'velocity': velocity,
        'attitude': attitude,
        'omega': omega
    }

In [7]:
async def get_trajectory_with_initial_position():
    state = await sim_step([0, 0, 0, 0])

    starting_point = TrajectoryPoint(
        time=state['time'],
        acceleration=state['acceleration'],
        attitude=state['attitude'],
        omega=state['omega'],
        position=state['position'],
        velocity=state['velocity'],
    )

    trajectory = Trajectory()
    trajectory.add_point(starting_point)

    return trajectory

In [8]:
trajectory_time_offset = 0
dp = np.zeros(3)

def get_next_trajectory_point(trajectory, sim_time, dt, dp):
    pt = trajectory.next_trajectory_point((sim_time + dt) / 1000.0)
    pt.position += dp
    return pt

In [9]:
from quadcopter_control.controller import QuadcopterController, calculate_drone_moment_of_inertia

In [10]:
DRONE_HEIGHT = 0.028
DRONE_RADIUS = 0.05
DRONE_MASS = 0.041

In [11]:
(Ixx, Iyy, Izz) = calculate_drone_moment_of_inertia(DRONE_HEIGHT, DRONE_RADIUS, DRONE_MASS)
(Ixx, Iyy, Izz)

(2.830366666666667e-05, 2.830366666666667e-05, 5.125000000000001e-05)

## Scenario -1: Determine Drone Motor / to Newton Rate

In [12]:
# drone_force = 9.81 * DRONE_MASS; drone_force

In [13]:
# positions = []
# forces = []

In [14]:
# for force in np.arange(50, 51, 0.01):
#     print("trying force", force)
#     for _ in range(50):
#         state = await sim_step([force, force, force, force])
#         forces.append(force)
#         positions.append(state['position'][2])

In [15]:
# plt.plot(positions[:1250])
# plt.plot(forces[:1250])
# plt.title("z", fontsize=20)
# plt.show()

In [16]:
scale = 0.40221 / (50.2*4); scale

0.0020030378486055778

500 = 1 newton

## Scenario 0: Zero Gains

Test whether controller runs without controller gains set.

In [17]:
trajectory = await get_trajectory_with_initial_position()

In [18]:
DT_MS = 15.625 # 64 hz

In [19]:
quadcopter_controller = QuadcopterController(
    DT_MS / 1e3,
    DRONE_MASS,
    Ixx, Iyy, Izz,
    l=0.05,
    max_tilt_angle=0.22,
    max_ascent_rate=20.0
)

In [20]:
trajectory = await get_trajectory_with_initial_position()


async def control_step(state):
    motors = quadcopter_controller.run_control(
        traj_pt=trajectory.next_trajectory_point((state['time'] + DT_MS*1e3)/1e6),
        est_pos=state['position'],
        est_vel=state['velocity'],
        est_omega=state['omega'],
        est_att=state['attitude'],
    ) 

    print(motors)

    return await sim_step(motors.tolist())

## Scenario 1: Altitude Control

Tune gains until drone stays in place. on a desired height.

In [21]:
state = await sim_step([0, 0, 0, 0])
trajectory = await get_trajectory_with_initial_position()

In [22]:
sim_time = (await sim_step([0, 0, 0, 0]))['time']
n_episodes = 800

for i in range(0, n_episodes):
    trajectory.add_point(TrajectoryPoint(
        time=(sim_time + (i*DT_MS*1e3) / 1e6),
        position=np.array([0.0, 0.0, 5.0]),
        velocity=np.zeros(3),
        acceleration=np.zeros(3),
        attitude=np.quaternion(1.0, 0.0, 0.0, 0.0),
        omega=np.array([0.1, 0.1, 0.1])
    ))

In [ ]:
import time

In [ ]:
async def tune_controller(
        kp_pqr=np.array([95.0, 95.0, 6.0]),
        kp_bank=0.0,
        kp_pos_z=0.0,
        kp_vel_z=0.0,
        ki_pos_z=0.0,
        kp_pos_xy=0.0,
        kp_yaw=0.0,
        kp_vel_xy=0.0,
        kappa=1.0
):
    global entity
    await sim.rpc("session.restart")
    entity = await sim.rpc("map.spawn_uav", {
        "robot_id": 0,
        "position": {
            "x": 0,
            "y": 0,
            "z": 0,
        },
    })
    time.sleep(1)
    await sim.rpc("session.run")

    quadcopter_controller.set_gains(
        kp_pqr=kp_pqr,
        kp_bank=kp_bank,
        kp_pos_z=kp_pos_z,
        kp_vel_z=kp_vel_z,
        ki_pos_z=ki_pos_z,
        kp_pos_xy=kp_pos_xy,
        kp_yaw=kp_yaw,
        kp_vel_xy=kp_vel_xy,
        kappa=kappa
    )

    positions = []

    state = await sim_step([0, 0, 0, 0])

    for _ in range(n_episodes):
        traj_pt = trajectory.next_trajectory_point(
            (state['time'] + DT_MS * 1e3) / 1e6)

        motors = quadcopter_controller.run_control(
            traj_pt,
            state['position'],
            state['velocity'],
            state['omega'],
            state['attitude'],
        ) / 0.02

        motors = np.clip(motors, 30, 100)


        positions.append(state['position'])

        state = await sim_step(motors.tolist())

    return np.array(positions)

In [24]:
(positions) = await tune_controller(
    kp_pqr=np.array([95.0, 95.0, 6.0]),
    ki_pos_z=55.0,
    kp_pos_z=0.0,
    kp_vel_z=0.00,
    kp_pos_xy=0.0,
    kp_vel_xy=0.0,
    kp_yaw=0.0,
    kp_bank=0.0
)

ReplyError: Error 100: entity 85v2 doesn't support actuator control (temporary: True)